### Clase 1: Introducción al cómputo en la nube

En esta clase veremos brevemente sobre lo que es y no es el cómputo en la nube. También, aprenderás a hacer tu cuenta en Azure y desplegarás tu primer modelo de Machine Learning en la nube.

Este archivo corresponde a esta última parte.

## 1.1 Experimentos de Machine Learning.

A continuación, presentamos un experimento pequeño sobre Machine Learning.

Entonces los guardamos con el formato Pickle (ADVERTENCIA, NO ES SEGURO)

In [12]:
import pickle

file = open('linear_regressor.pkl', 'wb')
pickle.dump(reg, file)
file.close()

# DESPLEGANDO EN LA NUBE.

Lo primero que debemos hacer es generar un espacio de trabajo (workspace), utilizando para ello el id de nuestra suscripción.

In [1]:
#GETTING MY ID:
import json

id = open('my_id.json', 'r')
mi = json.load(id)

from azureml.core import Workspace

ws = Workspace.create(name="testmodel",
                      subscription_id = mi["my_id"],
                      resource_group = "deploymodel",
                      location = "centralindia") #centralindia

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (numpy 1.26.4 (c:\users\felip\miniconda3\envs\mlearn\lib\site-packages), Requirement.parse('numpy!=1.19.4,<1.24; sys_platform == "win32"'), {'azureml-dataset-runtime'}).


Cleaning up past default Resource Group Deployments on the subscription to avoid limit of 10
Deleting past Resource Group Deployment with name: DeployResourceGroup-d87935a90b
Deploying KeyVault with name testmodekeyvault261f69e8.
Deploying StorageAccount with name testmodestoragee2091e308.
Deploying AppInsights with name testmodeinsights0194cc63.
Deployed AppInsights with name testmodeinsights0194cc63. Took 28.15 seconds.
Deploying Workspace with name testmodel.
Deployed Workspace with name testmodel. Took 41.73 seconds.


Posteriormente, registramos el modelo que vamos a utilizar. En este caso, el modelo en formato pickle. También hay que asignarle un nombre.

In [2]:
from azureml.core.model import Model

registered_model = Model.register(model_path="model.pkl",
                                  model_name="modelreg",
                                  workspace=ws)

Registering model modelreg


In [3]:
from azureml.core.environment import Environment
virtual_env = Environment("env")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    'pandas',
    'scikit-learn'
    ])

In [4]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="API/score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name='testmodel',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\Felip\AppData\Local\Temp\ipykernel_27584\979962562.py:10: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [ ]:
service.wait_for_deployment()

In [ ]:
scoring_uri = service.scoring_uri
# Save the uri to a file
with open('scoring_uri.txt', 'w') as f:
    f.write(scoring_uri)


## USING THE MODEL

In [ ]:
import pandas as pd
input_data = pd.read_csv("load_data/temp_data/output.csv")
    # Serialize the first 5 rows of the DataFrame to JSON
input_data = input_data.head(5).to_json(orient="records")
input_data = {
    "data": json.loads(input_data)  # Asegúrate de que sea una lista de registros
}
import json
two_houses_json = json.dumps({"data": [two_houses]})

headers = {"Content-Type": "application/json"}

import requests
response = requests.post(scoring_uri, 
                         data=two_houses_json, 
                         headers=headers)

if response.status_code == 200:
  result = json.loads(response.json())
  print(result)
  two_houses["median_house_value"] = result["result"]
  display(pd.DataFrame(two_houses))
else:
  print(f"Error: {response.text}")

{'result': [-12493018.054463292, -10942548.677924668]}


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-230.10,50.1,10,798,26,435346,123,2.325345,-1.249302e+07
1,-195.56,65.3,20,675,79,346234,4523,3.423424,-1.094255e+07
